install packages

In [ ]:
!pip install gradio

In [2]:
import pickle

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from joblib import load
import pickle
from scipy.sparse import vstack, hstack, csr_matrix
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

In [3]:
model = load('./ridge_model.pkl')


# PKLファイルから変換器を読み込みます
with open('./cv_cate.pkl', 'rb') as f:
    cv_cate = pickle.load(f)

with open('./cv_name.pkl', 'rb') as f2:
    cv_name = pickle.load(f2)

with open('./tv.pkl', 'rb') as e:
    tv = pickle.load(e)

with open('./lb.pkl', 'rb') as g:
    lb = pickle.load(g)

**アプリ**

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.3 MB/s et

雛形

In [17]:
# DeepL APIキーを設定します
import requests

API_KEY = "a1813ecb-cac9-4958-b1a2-5d5d653c46f3:fx"

# 日本語から英語への翻訳関数
def translate_to_english(japanese_text):
    # DeepL APIにリクエストを送信
    response = requests.post(
        "https://api-free.deepl.com/v2/translate",
        data={
            "text": japanese_text,
            "source_lang": "JA",
            "target_lang": "EN",
            "auth_key": API_KEY
        }
    )
    # レスポンスから翻訳されたテキストを取得
    translated_text = response.json()["translations"][0]["text"]
    return translated_text


print()

In [ ]:
import gradio as gr
import pickle
import pandas as pd



# Load the pre-trained model
# with open('path/to/your/model.pkl', 'rb') as model_file:
#     model = pickle.load(model_file)

def predict_price(name, item_condition_id, category_name, brand_name, price, shipping, item_description):
    # 日本語の入力を英語に翻訳
    name_en = translate_to_english(name)
    category_name_en = translate_to_english(category_name)
    brand_name_en = translate_to_english(brand_name)
    item_description_en = translate_to_english(item_description)

    # Create a DataFrame with the translated input data
    data = {
        'name': [name_en],
        'item_condition_id': [item_condition_id],
        'category_name': [category_name_en],
        'brand_name': [brand_name_en],
        'price': [price],
        'shipping': [shipping],
        'item_description': [item_description_en]
    }
    combined = pd.DataFrame(data)

    combined = combined.astype({'item_condition_id': 'category', 'category_name': 'category', 'brand_name': 'category'})
    combined['item_condition_id'] = pd.Categorical(combined['item_condition_id'], categories=[1, 2, 3, 4, 5], ordered=True)
    X_dummies = csr_matrix(pd.get_dummies(combined[['item_condition_id', 'shipping']], sparse=True).values)
    X_name = cv_name.transform(combined['name'])
    X_category = cv_cate.transform(combined['category_name'])

    X_description = tv.transform(combined['item_description'])
    X_brand = lb.transform(combined['brand_name'])
    sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()

    # Make predictions
    predictions = model.predict(sparse_merge)

    # Return the prediction
    return predictions

# Define the Gradio interface
interface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.components.Textbox(label="Name"),
        gr.components.Number(label="Item Condition ID"),
        gr.components.Textbox(label="Category Name"),
        gr.components.Textbox(label="Brand Name"),
        gr.components.Number(label="Price"),
        gr.components.Number(label="Shipping"),
        gr.components.Textbox(label="Item Description")
    ],
    outputs=gr.components.Textbox(label="Predicted Price")
)


# Webアプリを作成
#app = gr.Interface(fn=greet_english, inputs="text", outputs="text", title="English Greeting Translator")


# Launch the interface
if __name__ == "__main__":
    interface.launch(debug=True,share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://838c457cfba44dc1bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
